## 利用SVD简化数据
    将数据用其他的集中矩阵表示，简化了原来数据的存储方式。

### 矩阵分解（SVD矩阵分解）
    
在很多情况下,数据中的一小段携带了数据集中的大部分信息,其他信息则要么是噪声,要么就是毫不相关的信息。在线性代数中还有很多矩阵分解技术。 矩阵分解可以将原始矩阵表示成新的易于处理的形式。
SVD将原始数据集矩阵$Data_{m*n}$分解为三个矩阵：$$Data_{m*n}=Data1_{m*m}\sum_{m*n}Data2_{n*n}$$

矩阵${\sum}$：
+ 只有对角元素（也称奇异值），其他元素均为0 

+ 对角元素是从大到小排列的 

+ 奇异值就是矩阵$Data*Data^T$特征值的平方根

在某个奇异值(r个)之后, 其它的奇异值由于值太小,被忽略置为0, 这就意味着数据集中仅有r个重要特征,而其余特征都是噪声或冗余特征。如下图所示： 
![002](images/002.png)
如何选择数值r?

确定要保留的奇异值数目有很多启发式的策略,其中一个典型的做法就是保留矩阵中90%的能量信息.为了计算能量信息,将所有的奇异值求其平方和,从大到小叠加奇异值,直到奇异值之和达到总值的90%为止;另一种方法是,当矩阵有上万个奇异值时, 直接保留前2000或3000个.,但是后一种方法不能保证前3000个奇异值能够包含钱90%的能量信息,但是操作简单.

SVD分解很耗时,通过离线方式计算SVD分解和相似度计算,是一种减少冗余计算和推荐所需时间的办法.

*利用python计算SVD

Numpy的linalg的线性代数工具箱来实现，其有包括处理求范式、逆、行列式、伪逆、秩、qr分解、svd分解等等函数。

In [9]:
from numpy import *
from numpy import linalg as la

def loadExData():
    return [[1,1,1,0,0],
           [2,2,2,0,0],
           [5,5,5,0,0],
           [1,1,0,2,2],
           [0,0,0,3,3],
           [0,0,0,1,1]]
Data=loadExData()

U,Sigma,VT=la.svd(Data)
print(shape(VT))
print(Sigma)
#由于Sigma是以向量的形式存储，故需要将其转为矩阵，
sig3=mat([[Sigma[0],0,0],[0,Sigma[1],0],[0,0,Sigma[2]]])
#ig3=diag(Sigma)[:3,:3]
#重构原始矩阵
print(U[:,:3]*sig3*VT[:3,:])
print(shape(U[:,:3]*sig3*VT[:3,:]))
#重构矩阵和原始矩阵非常接近，类似于压缩图像，只保留图像分解后的两个方阵和一个对角阵的对角元素，就可以恢复原始图像。

(5, 5)
[9.56627891e+00 5.29323150e+00 6.84111192e-01 6.09875600e-16
 0.00000000e+00]
[[ 1.00000000e+00  1.00000000e+00  1.00000000e+00 -7.12646088e-16
  -7.38992201e-16]
 [ 2.00000000e+00  2.00000000e+00  2.00000000e+00 -4.67074296e-16
  -5.19766521e-16]
 [ 5.00000000e+00  5.00000000e+00  5.00000000e+00  5.44703171e-16
   3.85108612e-16]
 [ 1.00000000e+00  1.00000000e+00  3.88578059e-16  2.00000000e+00
   2.00000000e+00]
 [ 5.55111512e-17  5.82867088e-16 -3.33066907e-16  3.00000000e+00
   3.00000000e+00]
 [-1.38777878e-17  1.59594560e-16 -5.55111512e-17  1.00000000e+00
   1.00000000e+00]]
(6, 5)


## 基于协同过滤（collaborative filtering）的推荐引擎

近些年来，推荐引擎对因特网用户而言已经不是什么新鲜事物了，Amazon会根据顾客的购买历史向他们推荐物品，新闻网站会对用户推荐新闻报道等。

接下来我们介绍被称为“协同过滤”的方法，来实现推荐功能。协同过滤是通过将用户和其他用户的数据进行对比来实现推荐的。

数据组织成了类似图14-2所给出的矩阵的形式，当数据采用这种方式进行组织时，我们就可以比较用户或物品之间的相似度了，当知道相似度后，就可以利用已有的数据来预测未知用户喜好。

例如: 试图对某个用户喜欢的电影进行预测,搜索引擎会发现有一部电影该用户还没看过,然后它会计算该电影和用户看过的电影之间的相似度, 如果相似度很高, 推荐算法就会认为用户喜欢这部电影.

缺点: 在协同过滤情况下, 由于新物品到来时由于缺乏所有用户对其的喜好信息,因此无法判断每个用户对其的喜好.而无法判断某个用户对其的喜好,也就无法利用该商品.

#### 相似度计算

不利用专家给出的重要属性来描述物品从而计算其相似度，而是利用用户对他们的意见来计算相似度，这就是协同过滤使用的方法。并不关心物品的描述属性，而是严格按照许多用户的观点来计算相似度。

![003](images/003.png)

+ 1.欧氏距离
![004](images/004.png)

相似性度量方式： 0~1之间的变化，可以使用
相似度=\frac{1}{1+距离}来度量，距离为0时，相似度为1，距离很大时，相似度趋于0。

+ 2.皮尔逊相关系数（person correlation）

度量两个向量之间的相似度，相对于欧式距离的优势在于对用户评级的量级并不敏感。

numpy中实现：corrcoef()

取值范围：[-1,1]，通过$0.5+0.5∗corrcoef()$函数来计算，将其取值范围归一化到[0,1]。

+ 3.余弦相似度（cosine similarity）

度量两个向量夹角的余弦值，如果夹角为90°，则相似度为0，如果两个向量方向相同，相似度为1。

取值范围：[-1,+1]，因此也需要将其归一化到(0,1)之间。

定义：$cos{\theta}=\frac{A*B}{||A||*||B||}$

其中，||A||,||B||表示向量的2范数。

numpy中，范数的计算方法：linalg.norm()

范数=x到0点的欧氏距离

+ 其他距离
![005](images/005.png)

各个距离的计算方法：



In [10]:
# coding=utf-8
from numpy import *
from numpy import linalg as la
def loadExData():
    return [[1,1,1,0,0],
           [2,2,2,0,0],
           [5,5,5,0,0],
           [1,1,0,2,2],
           [0,0,0,3,3],
           [0,0,0,1,1]]

# 利用不同的方法计算相似度
def eulidSim(inA, inB):
    return 1.0/(1.0+la.norm(inA - inB))#根据欧式距离计算相似度

def pearsSim(inA, inB):
    if len(inA)<3:
        return 1.0
    else:
        return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA, inB):
    num   = float(inA.T*inB)            #向量inA和向量inB点乘,得cos分子
    denom = la.norm(inA)*la.norm(inB)   #向量inA,inB各自范式相乘，得cos分母
    return 0.5+0.5*(num/denom)          #从-1到+1归一化到[0,1]

myMat = mat(loadExData())
print(eulidSim(myMat[:,0], myMat[:,4])) #第一行和第五行利用欧式距离计算相似度
print(eulidSim(myMat[:,0], myMat[:,0])) #第一行和第一行欧式距离计算相似度
print(cosSim(myMat[:,0], myMat[:,4]))   #第一行和第五行利用cos距离计算相似度
print(cosSim(myMat[:,0], myMat[:,0]))   #第一行和第一行利用cos距离计算相似度
print(pearsSim(myMat[:,0], myMat[:,4])) #第一行和第五行利用皮尔逊距离计算相似度
print(pearsSim(myMat[:,0], myMat[:,0])) #第一行和第一行利用皮尔逊距离计算相似度

0.13507810593582123
1.0
0.5480015360737319
1.0
0.2041960108450192
1.0


餐馆菜肴推荐系统

+ 构建一个基本引擎，能够寻找用户没有品尝过的菜肴

+ 通过SVD来减少特征空间，并提高推荐效果

+ 将程序打包并通过用户可读的人机界面提供给人们使用



In [11]:
from numpy import *
from numpy import linalg as la

def loadExData2():
    return [[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
            [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
            [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
            [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
            [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
            [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
            [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
            [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
            [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
            [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
            [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

# 利用不同的方法计算相似度
def eulidSim(inA, inB):
    return 1.0/(1.0+la.norm(inA - inB))#根据欧式距离计算相似度

def pearsSim(inA, inB):
    if len(inA)<3:
        return 1.0
    else:
        return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA, inB):
    num   = float(inA.T*inB)            #向量inA和向量inB点乘,得cos分子
    denom = la.norm(inA)*la.norm(inB)   #向量inA,inB各自范式相乘，得cos分母
    return 0.5+0.5*(num/denom)          #从-1到+1归一化到[0,1]


def standEst(dataMat,user,simMeas,item):
    """
    计算在给定相似度计算方法的前提下，用户对物品的估计评分值
    :param dataMat: 数据矩阵
    :param user: 用户编号
    :param simMeas: 相似性度量方法
    :param item: 物品编号
    :return:
    """
    #数据中行为用于，列为物品，n即为物品数目
    n=shape(dataMat)[1]
    simTotal=0.0
    ratSimTotal=0.0
    #用户的第j个物品
    for j in range(n):
        userRating=dataMat[user,j]
        if userRating==0:
            continue
        #寻找两个用户都评级的物品
        overLap=nonzero(logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0))[0]

        if len(overLap)==0:
            similarity=0
        else:
            similarity=simMeas(dataMat[overLap,item],dataMat[overLap,j])

        simTotal+=similarity
        ratSimTotal+=simTotal*userRating

    if simTotal==0:
        return 0
    else:
        return ratSimTotal/simTotal

def recommend(dataMat,user,N=3,simMeas=cosSim,estMethod=standEst):
    """
    推荐引擎，会调用standEst()函数
    :param dataMat: 数据矩阵
    :param user: 用户编号
    :param N:前N个未评级物品预测评分值
    :param simMeas:
    :param estMethod:
    :return:
    """
    #寻找未评级的物品，nonzeros()[1]返回参数的某些为0的列的编号，dataMat中用户对某个商品的评价为0的列
    #矩阵名.A：将矩阵转化为array数组类型
    #nonzeros(a)：返回数组a中不为0的元素的下标
    unratedItems=nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems)==0:
        return 'you rated everything!'
    itemScores=[]
    for item in unratedItems:
        estimatedScore=estMethod(dataMat,user,simMeas,item)
        itemScores.append((item,estimatedScore))
    return sorted(itemScores,key=lambda  jj:jj[1],reverse=True)[:N]

#======================基于SVD的评分估计==========================
def SVDEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0
    ratSimTotal = 0.0
    U, Sigma, VT = la.svd(dataMat)
    Sig4 = mat(eye(4)*Sigma[:4]) #化为对角阵，或者用linalg.diag()函数可破
    xformedItems = dataMat.T*U[:,:4]*Sig4.I#构造转换后的物品
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j == item:
            continue
        similarity = simMeas(xformedItems[item,:].T, xformedItems[j, :].T)
        print("the %d and %d similarity is: %f" %(item,j,similarity))
        simTotal += similarity
        ratSimTotal += similarity*userRating
    if simTotal ==0 :
        return 0
    else:
        return ratSimTotal/simTotal

myMat = mat(loadExData2())
print(recommend(myMat, 1, estMethod = SVDEst))
print(recommend(myMat, 1, estMethod = SVDEst, simMeas = pearsSim))

the 0 and 3 similarity is: 0.490950
the 0 and 5 similarity is: 0.484274
the 0 and 10 similarity is: 0.512755
the 1 and 3 similarity is: 0.491294
the 1 and 5 similarity is: 0.481516
the 1 and 10 similarity is: 0.509709
the 2 and 3 similarity is: 0.491573
the 2 and 5 similarity is: 0.482346
the 2 and 10 similarity is: 0.510584
the 4 and 3 similarity is: 0.450495
the 4 and 5 similarity is: 0.506795
the 4 and 10 similarity is: 0.512896
the 6 and 3 similarity is: 0.743699
the 6 and 5 similarity is: 0.468366
the 6 and 10 similarity is: 0.439465
the 7 and 3 similarity is: 0.482175
the 7 and 5 similarity is: 0.494716
the 7 and 10 similarity is: 0.524970
the 8 and 3 similarity is: 0.491307
the 8 and 5 similarity is: 0.491228
the 8 and 10 similarity is: 0.520290
the 9 and 3 similarity is: 0.522379
the 9 and 5 similarity is: 0.496130
the 9 and 10 similarity is: 0.493617
[(4, 3.344714938469228), (7, 3.329402072452697), (9, 3.3281008763900686)]
the 0 and 3 similarity is: 0.341942
the 0 and 5 simila

![title](images/006.png)
![title](images/007.png)


In [15]:
from numpy import *
from numpy import linalg as la

def printMat(inMat,thresh=0.8):
    """
    打印矩阵
    :param inMat:输入数据集
    :param thresh:阈值
    :return:
    """
    #由于矩阵包含了浮点数，因此必须定义深色和浅色，通过阈值来界定
    #元素大于阈值，打印1，小于阈值，打印0
    for i in range(32):
        for k in range(32):
            if float(inMat[i,k])>thresh:
                print(1),
            else:
                print(0),
        print('')

def imgCompress(numSV=3,thresh=0.8):
    """
    实现图像的压缩，允许基于任意给定的奇异值数目来重构图像
    :param numSV:
    :param thresh:
    :return:
    """
    #构建列表
    myl=[]
    #打开文本文件，从文件中以数值方式读入字符
    for line in open("0_5.txt").readlines():
        newRow=[]
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat=mat(myl)
    print(myMat)

    print("******original matrix**********")
    #使用SVD对图像进行SVD分解和重构
    #printMat(myMat,thresh)
    U,Sigma,VT=la.svd(myMat)
    #建立一个全0矩阵
    SigRecon=mat(zeros((numSV,numSV)))
    #将奇异值填充到对角线
    for k in range(numSV):
        SigRecon[k,k]=Sigma[k]
    #重构矩阵
    reconMat=U[:,:numSV]*SigRecon*VT[:numSV,:]
    print(shape(reconMat))
    print("***reconstructed matrix using %d singular values***" %numSV)
    #printMat(reconMat,thresh)

if __name__=='__main__':
    imgCompress(2)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
******original matrix**********
(32, 32)
***reconstructed matrix using 2 singular values***
